In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import f1_score, accuracy_score
from sklearn.linear_model import LogisticRegression
import numpy as np
from tqdm import tqdm
from sklearn.model_selection import train_test_split
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class RNNClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(RNNClassifier, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)
        out, _ = self.rnn(x, h0)
        out = out[:, -1, :]
        out = self.fc(out)
        return out

def load_data(file_name):
    with np.load(file_name) as data:
        data_array = data['data']
        labels_array = data['labels']
    return data_array, labels_array

def to_tensor(data, labels):
    data_tensor = torch.Tensor(data)
    labels_tensor = torch.LongTensor(labels.argmax(axis=1))
    return data_tensor, labels_tensor

train_data, train_labels = load_data('trainset_normalized.npz')
test_data, test_labels = load_data('testset_normalized.npz')

train_data_tensor, train_labels_tensor = to_tensor(train_data, train_labels)
test_data_tensor, test_labels_tensor = to_tensor(test_data, test_labels)

train_X, val_X, train_y, val_y = train_test_split(train_data_tensor, train_labels_tensor, test_size=0.2)

batch_size = 32
train_loader = DataLoader(TensorDataset(train_X, train_y), batch_size=batch_size, shuffle=True)

input_size = 16 
hidden_size = 256
output_size = 5 
num_layers = 3 
learning_rate = 0.0003242487387355423
num_epochs = 10
num_models = 5

models = [RNNClassifier(input_size, hidden_size, output_size, num_layers).to(device) for _ in range(num_models)]

base_model_predictions = []
for model in models:
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)
    criterion = nn.CrossEntropyLoss()
    for epoch in tqdm(range(num_epochs), desc="Training model"):
        model.train()
        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

    model.eval()
    predictions = []
    with torch.no_grad():
        for inputs, _ in DataLoader(TensorDataset(val_X, val_y), batch_size=batch_size):
            inputs = inputs.to(device)
            outputs = model(inputs)
            _, predicted = torch.max(outputs, 1)
            predictions.extend(predicted.cpu().numpy())
    base_model_predictions.append(predictions)


stacked_predictions = np.column_stack(base_model_predictions)
meta_model = LogisticRegression()
meta_model.fit(stacked_predictions, val_y.numpy())
final_predictions = meta_model.predict(stacked_predictions)

accuracy = accuracy_score(val_y.numpy(), final_predictions)
f1 = f1_score(val_y.numpy(), final_predictions, average='weighted')

print(f'Stacked Model Accuracy: {accuracy:.4f}')
print(f'Stacked Model F1 Score: {f1:.4f}')

Training model:  10%|█         | 1/10 [00:39<05:52, 39.15s/it]


KeyboardInterrupt: 

Ensemble Accuracy: 0.6857
Ensemble F1 Score: 0.6444
